**End-to-end Machine Learning**

Borrows very heavily from _Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow_ by Aurelien Geron

# Setup

In [ ]:
# Common imports
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns

# Display all columns
pd.options.display.max_columns = 200

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "image_outputs")

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Get the data

In [ ]:
EARTHQUAKE_PATH = os.path.join("datasets")

In [ ]:
def load_data(data_path=EARTHQUAKE_PATH, file_name = "train_values.csv"):
    csv_path = os.path.join(data_path, file_name)
    return pd.read_csv(csv_path)

In [ ]:
raw_values = load_data()
raw_labels = load_data(file_name="train_labels.csv")
raw_data = pd.merge(raw_labels,raw_values,on='building_id')
raw_data.shape

# Take a quick look at the data

There are 260,601 instances in the dataset, if you want to do a quick explore, I'd recommend using a sample.

In [ ]:
raw_data = raw_data.sample(frac=0.5)
raw_data.reset_index(drop=True,inplace=True)
raw_data.shape

Let’s take a look at the top five rows using the DataFrame’s head() method

In [ ]:
raw_data.head()

Each row represents a building. 

The `damage_grade` field is what we want to be able to predict, our _label_. There are 38 attributes, or _features_ relating to geographic area, house characteristics: age, type of roof, use, etc.

Let's take a closer look at these attributes:

In [ ]:
raw_data.info()

Great! There are no null values. Watch out, as this might be because missing values have not been encoded correctly (e.g. entered as 'missing' or blanks). 

In our data, most attributes seem to be numerical, but on closer inspection, the fields such as `has_secondary_use_other` are essentially boolean values. They have been already converted from objects, using a method called 'one hot encoding' which I will go into later.

In [ ]:
pre_encoded_features = [s for s in raw_data.columns if 'has_' in s]
raw_data[pre_encoded_features] = raw_data[pre_encoded_features].astype('int8')

Some fields, such as the fields ending in `_type` are objects. This 'object' type could mean a variety of types, such as a string, or text attribute. Here, they refer to distinct categories. You can find out what categories exist and how many buildings belong to each category by using the `value_counts()` method.

In [ ]:
raw_data["roof_type"].value_counts()

Another quick way to get a feel of the type of data you are dealing with is to plot a histogram for each numerical attribute.

In [ ]:
raw_data.drop('building_id',axis=1).select_dtypes(include=('int64')).hist(bins=50, figsize=(20,15))
plt.show()
save_fig("attribute_histogram_plots")

In [ ]:
categorical_features = list(raw_data.select_dtypes(include=['object']).columns)
numerical_features = ['area_percentage','height_percentage']
discrete_features = list(raw_data.drop(columns=categorical_features+numerical_features+pre_encoded_features+['building_id']).columns)

For categorical features, `countplot()` can be useful:

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))
for variable, subplot in zip(categorical_features, ax.flatten()):
    sns.countplot(raw_data[variable], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.show()
save_fig("cat_attribute_countplots")

If we take a closer look at our _label_: `damage_grade`:

In [ ]:
raw_data['damage_grade'] = raw_data['damage_grade'].astype('str')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
sns.countplot(raw_data['damage_grade'], ax=ax)
plt.show()
save_fig("class_balance_countplot")

In [ ]:
raw_data.damage_grade.value_counts(normalize=True)

In [ ]:
raw_data.damage_grade.value_counts()

The data is imbalanced with 33% ‘high’ damage grade,10% of ‘Low’ damage grade and 57% of ‘Medium’ damage grade. These imbalances can be adjusted at the modelling stage, using `class_weight`, but for clarity I am manually adjusting the class imbalance by taking an equal amount randomly from each class.

In [ ]:
train_high=raw_data[raw_data.damage_grade==3]
train_medium=raw_data[raw_data.damage_grade==2]
train_low=raw_data[raw_data.damage_grade==1]
train_high=train_high.sample(25000)
train_medium=train_medium.sample(25000)
train_low=train_low.sample(25000)
resampled_data = pd.concat([train_high, train_medium,train_low], ignore_index=True, sort =False)
resampled_data=resampled_data.sample(frac=1).reset_index(drop=True)
training_data = resampled_data.copy()
training_data = raw_data.copy()
resampled_data.head(5)

Let's take a quick look at the `age` feature:

In [ ]:
training_data.age.hist(bins=50, figsize=(10,5))
plt.title('Building Age Histogram (Before)')
save_fig("age_hist_plot_pre_cleaning")
plt.show()

What's going on >200?

In [ ]:
training_data_age_greater_200 = training_data[training_data.age>200]
sns.countplot(x ='age', data= training_data_age_greater_200, hue='damage_grade')
save_fig('age_outliers_countplot')
plt.show()

Uh oh, looks like we have some outliers. This seems like something we can exclude, as it is unlikely 1400 buildings are exactly 999 years old.

In [ ]:
age_index_to_drop = training_data[training_data['age'] > 200 ].index
 
# Delete these row indexes from dataFrame
training_data.drop(age_index_to_drop , inplace=True)
training_data.reset_index(drop=True,inplace=True)

Ideally, we would add this step to our processing function as part of our pipeline, which I will show you late

In [ ]:
training_data.age.hist()
save_fig('age_hist_plot_post_cleaning')
plt.title('Building Age Histogram (After)')
plt.show()

Let's have a look at how damage varies with age

In [ ]:
plt.figure(figsize=(18,8))
plt.subplot(211)
plt.title('Damage snapshot properties with less than 50 years old')
tr=training_data[training_data['age']<=50]
sns.countplot(x='age',data=tr,hue='damage_grade')
save_fig('age_countplot')
plt.show()

For the purposes of this notebook, I am going to drop `building_id` at this stage, using the index as identifier. In a real project, you may need to retain these for use later on

In [ ]:
building_ids = training_data.pop('building_id')

## Create a test set

Your brain is an amazing pattern detection system, which means that it is highly prone to overfitting: if you look at the test set, you may stumble upon some seemingly interesting pattern in the test data that leads you to select a particular kind of Machine Learning model. When you estimate the generalization error using the test set, your estimate will be too optimistic, and you will launch a system that will not perform as well as expected. This is called _data snooping_ bias.

Scikit-Learn provides a few functions to split datasets into multiple subsets in various ways. The simplest function is `train_test_split()`.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(training_data, test_size=0.2, random_state=42)

Random sampling methods are generally fine if your dataset is large enough, but if it is not, you run the risk of introducing a significant sampling bias. 

## Stratified sampling for ML

You chat with experts, who say that the age of the building is an important feature to predict damage. You may want to ensure that the test set is representative of the various ages in the whole dataset. Since the age income is a discrete numerical attribute, you first need to create an age category attribute. 

Most age values are clustered around 0-25 but some are much older. We need a sufficient number of instances in your dataset for each stratum, or the estimate of a stratum’s importance may be biased. We should not have too many strata, and each stratum should be large enough.

In [ ]:
training_data["age_cat"] = pd.cut(training_data["age"],
                               bins=[-np.inf, 5, 15,25, 40, np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
training_data["age_cat"].hist()
plt.title('Building age grouped into buckets')
save_fig('age_bucket_histogram')
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(training_data, training_data["age_cat"]):
    strat_train_set = training_data.loc[train_index]
    strat_test_set = training_data.loc[test_index]

In [ ]:
strat_test_set["age_cat"].value_counts(ascending=True) / len(strat_test_set)

In [ ]:
training_data["age_cat"].value_counts(ascending=True) / len(training_data)

In [ ]:
def age_cat_proportions(data):
    return data["age_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(training_data, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": age_cat_proportions(training_data),
    "Stratified": age_cat_proportions(strat_test_set),
    "Random": age_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

You can see here that the incremental gains are very small, as my dataset is so large

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("age_cat", axis=1, inplace=True)

# Discover and visualize the data to gain insights

In [ ]:
earthquake = train_set.copy()

Here I compute the standard correlation coefficient between every pair of attributes using the `corr()` method:

In [ ]:
corr_matrix = earthquake.corr()
g = sns.heatmap(corr_matrix, center=0,
            square=True, linewidths=0.01, cbar_kws={"shrink": .5}, annot=False, yticklabels=True,
                xticklabels=True,cmap=sns.diverging_palette(20, 220, n=200))
sns.despine()
g.figure.set_size_inches(20,10)
save_fig('correlation_matrix')
plt.show()

The feature `has_superstructure_mud_mortar_stone` seems to be the feature most positively correlated with our label, `damage_grade`. Let's explore this some more:

In [ ]:
g = sns.catplot(x="count_floors_pre_eq", y="age", col="has_superstructure_mud_mortar_stone", hue="damage_grade",data=earthquake)
save_fig('mud_mortar_stone_catplot')
plt.show()

One last thing you may want to do before preparing the data for Machine Learning algorithms is to try out various attribute combinations:

In [ ]:
earthquake["families_per_floor"] = earthquake["count_families"]/earthquake["count_floors_pre_eq"]
earthquake["floors_per_area"] = earthquake["count_floors_pre_eq"]/earthquake["area_percentage"]
earthquake["height_per_area"]=earthquake["height_percentage"]/earthquake["area_percentage"]

In [ ]:
corr_matrix = earthquake.corr()
corr_matrix["damage_grade"].sort_values(ascending=False)[:5]

This round of exploration does not have to be absolutely thorough; the point is to start off on the right foot and quickly gain insights that will help you get a first reasonably good prototype. But this is an iterative process: once you get a prototype up and running, you can analyze its output to gain more insights and come back to this exploration step.

# Prepare the data for Machine Learning algorithms

Instead of doing this manually, you should write functions for this purpose, for several good reasons:

- This will allow you to reproduce these transformations easily on any dataset (e.g., the next time you get a fresh dataset).

- You will gradually build a library of transformation functions that you can reuse in future projects.

- This will make it possible for you to easily try various transformations and see which combination of transformations works best.

In [ ]:
earthquake = train_set.drop(["damage_grade"], axis=1) # drop labels for training set
earthquake_labels = train_set["damage_grade"].copy()

## Data Cleaning

Most Machine Learning algorithms cannot work with missing features, so let’s create a few functions to take care of them. We are lucky enough to be working with a pre-cleaned dataset, with no missing values, but this is very unusual, and we could get some new data that has missing values. You normally have three options:

1. Get rid of the corresponding districts.

2. Get rid of the whole attribute.

3. Set the values to some value (zero, the mean, the median, etc.).

You can accomplish these easily using DataFrame’s `dropna()`, `drop()`, and `fillna()` methods

Scikit-Learn provides a handy class to take care of missing values: `SimpleImputer`. 

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Since the median can only be computed on numerical attributes, you need to create a copy of the data without categorical features:

In [ ]:
earthquake_num = earthquake.drop(categorical_features, axis=1)
imputer.fit(earthquake_num)

Now you can use this “trained” imputer to transform the training set by replacing missing values with the learned medians:

In [ ]:
X = imputer.transform(earthquake_num)

earthquake_tr = pd.DataFrame(X, columns=earthquake_num.columns,
                          index=earthquake_num.index)

## Handling Text and Categorical Attributes

So far we have only dealt with numerical attributes, but now let’s look at text attributes. In this dataset, there are a few: `land_surface_condition`, `foundation_type`, `roof_type`, `ground_floor_type`, `other_floor_type`, `position`, `plan_configuration` and `legal_ownership_status`. 

In [ ]:
earthquake_cat = earthquake[categorical_features]
earthquake_cat.head()

We could convert these features to numbers, but one issue with this representation is that ML algorithms will assume that two nearby values are more similar than two distant values. This may be fine in some cases (e.g., for ordered categories such as “bad,” “average,” “good,” and “excellent”).

Instead, let's create one binary attribute per category. This is called one-hot encoding, because only one attribute will be equal to 1. The new attributes are sometimes called _dummy_ attributes. Scikit-Learn provides a `OneHotEncoder` class to convert categorical values into one-hot vectors

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse=False)
earthquake_cat_1hot = cat_encoder.fit_transform(earthquake_cat)
earthquake_cat_1hot

Although Scikit-Learn provides many useful transformers, you will need to write your own for tasks such as custom cleanup operations or combining specific attributes. 

You will want your transformer to work seamlessly with Scikit-Learn functionalities (such as pipelines).

For example, here is a small transformer class that adds the combined attributes we discussed earlier:

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def add_extra_features(X):

    families_per_floor = X[:, families_ix] / X[:, floors_ix]
    floors_per_area = X[:, floors_ix] / X[:, area_ix]
    height_per_area = X[:, height_ix] / X[:, area_ix]
    return np.c_[X, families_per_floor, floors_per_area,
                     height_per_area]

## Feature Scaling

One of the most important transformations you need to apply to your data is feature scaling. Most Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales. 

There are two common ways to get all attributes to have the same scale: 
- min-max scaling
- standardization

Min-max scaling (many people call this normalization) is the simplest: values are shifted and rescaled so that they end up ranging from 0 to 1. We do this by subtracting the min value and dividing by the max minus the min. 

Standardization first subtracts the mean value and then divides by the standard deviation. Unlike min-max scaling, standardization does not bound values to a specific range, which may be a problem for some algorithms.

## Transformation Pipelines

As you can see, there are many data transformation steps that need to be executed in the right order. Fortunately, Scikit-Learn provides the Pipeline class to help with such sequences of transformations. Here is a small pipeline for the numerical attributes:

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
    ])

earthquake_num = earthquake.select_dtypes(['int8','int64'])
families_ix, floors_ix, area_ix, height_ix = [
    list(earthquake_num.columns).index(col)
    for col in ("count_families", "count_floors_pre_eq", "area_percentage", "height_percentage")]
earthquake_num_tr = num_pipeline.fit_transform(earthquake_num)

So far, we have handled the categorical columns and the numerical columns separately. It would be more convenient to have a single transformer able to handle all columns, applying the appropriate transformations to each column. `ColumnTransformer` works great for this:

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(earthquake_num.columns)
cat_attribs = categorical_features

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

earthquake_prepared = full_pipeline.fit_transform(earthquake)

# Select and train a model 

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(earthquake_prepared, earthquake_labels)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = earthquake.iloc[:5]
some_labels = earthquake_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", log_reg.predict(some_data_prepared))

Compare against the actual values:

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import f1_score

earthquake_predictions = log_reg.predict(earthquake_prepared)
log_f1 = f1_score(earthquake_labels, earthquake_predictions, average='micro')
log_f1

When this happens it can mean that the features do not provide enough information to make good predictions, or that the model is not powerful enough. As we saw in the previous chapter, the main ways to fix underfitting are to select a more powerful model, to feed the training algorithm with better features, or to reduce the constraints on the model. This model is not regularized, which rules out the last option. You could try to add more features (e.g., the log of the population), but first let’s try a more complex model to see how it does.

Let’s train a `DecisionTreeRegressor`. This is a powerful model, capable of finding complex nonlinear relationships in the data

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(earthquake_prepared, earthquake_labels)

In [ ]:
earthquake_predictions = tree_clf.predict(earthquake_prepared)
tree_f1 = f1_score(earthquake_labels, earthquake_predictions,average='micro')
tree_f1

Wait, what!? Could this model really be almost absolutely perfect? Of course, it is much more likely that the model has badly overfit the data. How can you be sure? As we saw earlier, you don’t want to touch the test set until you are ready to launch a model you are confident about, so you need to use part of the training set for training and part of it for model validation.

## Better evaluation using cross-validation

On option is Scikit-Learn’s K-fold cross-validation feature. The following code randomly splits the training set into 10 distinct subsets called folds, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation every time and training on the other 9 folds. The result is an array containing the 10 evaluation scores:

In [ ]:
from sklearn.model_selection import cross_val_score

tree_f1_scores = cross_val_score(tree_clf, earthquake_prepared, earthquake_labels,
                         scoring="f1_micro", cv=10)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_f1_scores)

Now the Decision Tree doesn’t look as good as it did earlier. 

Let’s compute the same scores for the Logistic Regression model just to be sure:

In [ ]:
log_scores = cross_val_score(log_reg, earthquake_prepared, earthquake_labels,
                             scoring="f1_micro", cv=10)
display_scores(log_scores)

Let’s try one last model now: the RandomForestRegressor. As we will see in Chapter 7, Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions. Building a model on top of many other models is called Ensemble Learning, and it is often a great way to push ML algorithms even further. We will skip most of the code since it is essentially the same as for the other models:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
forest_clf.fit(earthquake_prepared, earthquake_labels)

In [ ]:
forest_scores = cross_val_score(forest_clf, earthquake_prepared, earthquake_labels,
                                scoring="f1_micro", cv=10)
display_scores(forest_scores)

# Fine-tune your model

## Grid Search

Scikit-Learn’s `GridSearchCV` searches among the hyperparameters you want it to experiment with, using values you choose. It will use cross-validation to evaluate all the possible combinations of hyperparameter values.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×3) combinations of hyperparameters
    {'n_estimators': [30, 50, 100], 'max_features': [6,8,10]}
  ]

forest_clf = RandomForestClassifier(random_state=42)
# train across 5 folds, that's a total of (9)*5=45 rounds of training 
grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                           scoring='f1_micro', return_train_score=True)
grid_search.fit(earthquake_prepared, earthquake_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

Let's look at the score of each hyperparameter combination tested during the grid search:

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

## Analyze the Best Models and Their Errors

You will often gain good insights on the problem by inspecting the best models. For example, the `RandomForestClassifier` can indicate the relative importance of each attribute for making accurate predictions:

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["families_per_floor", "floors_per_area", "height_per_area"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## Evaluate Your System on the Test Set

After tweaking your models for a while, you eventually have a system that performs sufficiently well. Now is the time to evaluate the final model on the test set. There is nothing special about this process; just get the predictors and the labels from your test set, run your full_pipeline to transform the data and evaluate the final model on the test set:

In [ ]:
final_model = grid_search.best_estimator_

X_test = test_set.drop(["damage_grade"], axis=1)
y_test = test_set["damage_grade"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_f1 = f1_score(y_test, final_predictions, average='micro')

In [ ]:
final_f1

## A full pipeline with both preparation and prediction

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("random_forest", final_model)
])

full_pipeline_with_predictor.fit(earthquake, earthquake_labels)
print("Predictions:", full_pipeline_with_predictor.predict(some_data))
print("Labels:", list(some_labels))

## Model persistence using joblib

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
import joblib
joblib.dump(my_model, "my_model.pkl") 
my_model_loaded = joblib.load("my_model.pkl")

### Possible additions:
- Transformation to select k best features
- More models (+ ensemble models)
- RandomisedSearchCV
- GridSearch to find optimum imputation strategy

The grid search approach is fine when you are exploring relatively few combinations, like in the previous example, but when the hyperparameter search space is large, it is often preferable to use RandomizedSearchCV instead. This class can be used in much the same way as the GridSearchCV class, but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits:

If you let the randomized search run for, say, 1,000 iterations, this approach will explore 1,000 different values for each hyperparameter (instead of just a few values per hyperparameter with the grid search approach).